In [39]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import community as community_louvain  
import ast

# Carica dati
edges = pd.read_csv('dataset/spoti/edges.csv')
nodes = pd.read_csv('dataset/spoti/nodes.csv')
nodes_unique = nodes.drop_duplicates(subset=['spotify_id'], keep='first')


In [40]:
def parse_genres(genres_field):
    """Converte stringa rappresentante lista in lista Python"""
    if pd.isna(genres_field):
        return []
    if isinstance(genres_field, str):
        try:
            parsed = ast.literal_eval(genres_field)
            return parsed if isinstance(parsed, list) else []
        except:
            return []
    if isinstance(genres_field, list):
        return genres_field
    return []



In [41]:
# Parametri di selezione della rete
TOPLIST = 100  # Numero di artisti da selezionare
SELECTION_CRITERION = "degree"  # Opzioni: "degree", "popularity", "betweenness", "closeness"
POPULARITY_FIELD = "popularity"  # Campo popolarità nel dataframe nodes_ita

In [42]:
def is_italian(genres):
    if isinstance(genres, str) and 'ital' in genres.lower():
        return True
    return False

nodes_ita = nodes_unique[nodes_unique.apply(lambda row: is_italian(row['genres']), axis=1)]
nodes_ita['genres'] = nodes_ita['genres'].apply(parse_genres)
print(f"✓ Generi convertiti: {sum(nodes_ita['genres'].apply(len) > 0)} artisti con genere")
italian_ids = set(nodes_ita['spotify_id'])
print(f"Artisti italiani identificati: {len(italian_ids)}")


✓ Generi convertiti: 1073 artisti con genere
Artisti italiani identificati: 1073


C:\Users\Federico\AppData\Local\Temp\ipykernel_14368\3788033907.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nodes_ita['genres'] = nodes_ita['genres'].apply(parse_genres)


In [43]:
# Filtra solo collaborazioni fra artisti italiani
tt = edges[edges['id_0'].isin(italian_ids) & edges['id_1'].isin(italian_ids)]

# Creazione grafo completo
G_it = nx.Graph()
G_it.add_edges_from(tt[['id_0', 'id_1']].values)

# Aggiungi attributi dei nodi
attr_dict = nodes_ita.set_index('spotify_id').to_dict('index')
nx.set_node_attributes(G_it, attr_dict)

print(f"\n[GRAFO COMPLETO]")
print(f"Nodi (artisti italiani): {G_it.number_of_nodes()}")
print(f"Archi (collaborazioni): {G_it.number_of_edges()}")


[GRAFO COMPLETO]
Nodi (artisti italiani): 835
Archi (collaborazioni): 3070


In [44]:
def select_top_nodes(graph, nodes_df, n=100, criterion="degree", popularity_field="popularity"):
    """
    Seleziona i top n nodi secondo diversi criteri di centralità.
    
    Parametri:
    - graph: grafo NetworkX
    - nodes_df: dataframe con attributi degli artisti
    - n: numero di nodi da selezionare
    - criterion: metrica di selezione
    - popularity_field: nome colonna popolarità
    
    Returns:
    - Lista di nodi selezionati
    """
    
    if criterion == "degree":
        # Centralità di grado: numero di collaborazioni
        metric = dict(graph.degree())
        
    elif criterion == "popularity":
        # Popolarità su Spotify
        metric = {}
        for node in graph.nodes():
            if node in nodes_df['spotify_id'].values:
                pop = nodes_df.loc[nodes_df['spotify_id'] == node, popularity_field].values
                metric[node] = pop[0] if len(pop) > 0 else 0
            else:
                metric[node] = 0
                
    elif criterion == "betweenness":
        # Centralità di intermediazione: broker tra comunità
        metric = nx.betweenness_centrality(graph)
        
    elif criterion == "closeness":
        # Centralità di vicinanza: distanza media dagli altri
        metric = nx.closeness_centrality(graph)
        
    elif criterion == "eigenvector":
        # Centralità di autovettore: connessione a nodi importanti
        try:
            metric = nx.eigenvector_centrality(graph, max_iter=1000)
        except:
            print("Eigenvector centrality non convergente, uso degree")
            metric = dict(graph.degree())
    else:
        raise ValueError(f"Criterio '{criterion}' non valido")
    
    # Ordina e seleziona top n
    sorted_nodes = sorted(metric.items(), key=lambda x: x[1], reverse=True)
    top_nodes = [n for n, v in sorted_nodes[:n]]
    
    print(f"\n[SELEZIONE TOP {n}]")
    print(f"Criterio: {criterion}")
    print(f"Top 5 artisti selezionati:")
    for i, (node, value) in enumerate(sorted_nodes[:5], 1):
        name = graph.nodes[node].get('name', node)
        print(f"  {i}. {name} (valore: {value:.3f})")
    
    return top_nodes

# Seleziona i top artisti
top_nodes = select_top_nodes(G_it, nodes_ita, n=TOPLIST, criterion=SELECTION_CRITERION)

# Crea sottografo con solo i top artisti e le loro collaborazioni
G_top = G_it.subgraph(top_nodes).copy()

print(f"\n[GRAFO TOP {TOPLIST}]")
print(f"Nodi: {G_top.number_of_nodes()}")
print(f"Archi: {G_top.number_of_edges()}")



[SELEZIONE TOP 100]
Criterio: degree
Top 5 artisti selezionati:
  1. Guè (valore: 92.000)
  2. Night Skinny (valore: 75.000)
  3. Gemitaiz (valore: 70.000)
  4. Don Joe (valore: 68.000)
  5. Emis Killa (valore: 65.000)

[GRAFO TOP 100]
Nodi: 100
Archi: 918


In [45]:
print("\n" + "="*60)
print("1. PROPRIETÀ GLOBALI DELLA RETE")
print("="*60)

# Densità: rapporto tra archi esistenti e possibili
density = nx.density(G_top)
print(f"\nDensità: {density:.4f}")
print(f"  → La rete è {'molto' if density > 0.1 else 'poco'} connessa")

# Componenti connesse
components = list(nx.connected_components(G_top))
print(f"\nComponenti connesse: {len(components)}")
print(f"  → Dimensione componente principale: {len(max(components, key=len))} nodi")

# Lavora sulla componente gigante
G_main = G_top.subgraph(max(components, key=len)).copy()

# Diametro e cammino medio (solo se connesso)
if nx.is_connected(G_main):
    diameter = nx.diameter(G_main)
    avg_path = nx.average_shortest_path_length(G_main)
    print(f"\nDiametro: {diameter}")
    print(f"  → Massima distanza tra due artisti: {diameter} passaggi")
    print(f"\nCammino medio: {avg_path:.3f}")
    print(f"  → Distanza media tra artisti: {avg_path:.3f} collaborazioni")
else:
    print("\nGrafo non completamente connesso, diametro non calcolabile")

# Coefficiente di clustering
clustering_coeff = nx.average_clustering(G_main)
print(f"\nCoefficient di clustering medio: {clustering_coeff:.4f}")
print(f"  → I collaboratori di un artista {'spesso' if clustering_coeff > 0.3 else 'raramente'} collaborano tra loro")



1. PROPRIETÀ GLOBALI DELLA RETE

Densità: 0.1855
  → La rete è molto connessa

Componenti connesse: 1
  → Dimensione componente principale: 100 nodi

Diametro: 5
  → Massima distanza tra due artisti: 5 passaggi

Cammino medio: 2.010
  → Distanza media tra artisti: 2.010 collaborazioni

Coefficient di clustering medio: 0.3699
  → I collaboratori di un artista spesso collaborano tra loro


In [31]:
print("\n" + "="*60)
print("2. DISTRIBUZIONE DEI GRADI")
print("="*60)

# Calcola gradi
degrees = [G_main.degree(n) for n in G_main.nodes()]
degree_count = Counter(degrees)

# Statistiche
avg_degree = np.mean(degrees)
median_degree = np.median(degrees)
max_degree = max(degrees)

print(f"\nGrado medio: {avg_degree:.2f}")
print(f"Grado mediano: {median_degree:.0f}")
print(f"Grado massimo: {max_degree}")

# Trova artisti più connessi
top_degree = sorted(G_main.degree(), key=lambda x: x[1], reverse=True)[:10]
print(f"\nTop 10 artisti per numero di collaborazioni:")
for i, (node, deg) in enumerate(top_degree, 1):
    name = G_main.nodes[node].get('name', node)
    print(f"  {i}. {name}: {deg} collaborazioni")

# Test distribuzione power-law
print(f"\nLa distribuzione {'segue' if max_degree > 5*avg_degree else 'non segue'} un pattern scale-free")
print(f"  → {'Pochi hub dominano' if max_degree > 5*avg_degree else 'Rete più omogenea'}")



2. DISTRIBUZIONE DEI GRADI

Grado medio: 18.36
Grado mediano: 17
Grado massimo: 57

Top 10 artisti per numero di collaborazioni:
  1. Guè: 57 collaborazioni
  2. Gemitaiz: 49 collaborazioni
  3. Night Skinny: 44 collaborazioni
  4. Emis Killa: 44 collaborazioni
  5. Fabri Fibra: 39 collaborazioni
  6. Jake La Furia: 38 collaborazioni
  7. Marracash: 37 collaborazioni
  8. Don Joe: 35 collaborazioni
  9. MadMan: 34 collaborazioni
  10. Lazza: 32 collaborazioni

La distribuzione non segue un pattern scale-free
  → Rete più omogenea


In [32]:
print("\n" + "="*60)
print("3. MISURE DI CENTRALITÀ")
print("="*60)

# Calcola diverse centralità
degree_cent = nx.degree_centrality(G_main)
betweenness_cent = nx.betweenness_centrality(G_main)
closeness_cent = nx.closeness_centrality(G_main)

try:
    eigenvector_cent = nx.eigenvector_centrality(G_main, max_iter=1000)
except:
    eigenvector_cent = {n: 0 for n in G_main.nodes()}

# Crea dataframe comparativo
centrality_df = pd.DataFrame({
    'artist': [G_main.nodes[n].get('name', n) for n in G_main.nodes()],
    'degree': [degree_cent[n] for n in G_main.nodes()],
    'betweenness': [betweenness_cent[n] for n in G_main.nodes()],
    'closeness': [closeness_cent[n] for n in G_main.nodes()],
    'eigenvector': [eigenvector_cent[n] for n in G_main.nodes()]
})

print("\n[DEGREE CENTRALITY] - Artisti più connessi:")
top_deg = centrality_df.nlargest(5, 'degree')
for idx, row in top_deg.iterrows():
    print(f"  {row['artist']}: {row['degree']:.4f}")

print("\n[BETWEENNESS CENTRALITY] - Artisti bridge tra comunità:")
top_bet = centrality_df.nlargest(5, 'betweenness')
for idx, row in top_bet.iterrows():
    print(f"  {row['artist']}: {row['betweenness']:.4f}")

print("\n[CLOSENESS CENTRALITY] - Artisti centrali nella rete:")
top_clo = centrality_df.nlargest(5, 'closeness')
for idx, row in top_clo.iterrows():
    print(f"  {row['artist']}: {row['closeness']:.4f}")

print("\n[EIGENVECTOR CENTRALITY] - Artisti connessi ad altri importanti:")
top_eig = centrality_df.nlargest(5, 'eigenvector')
for idx, row in top_eig.iterrows():
    print(f"  {row['artist']}: {row['eigenvector']:.4f}")



3. MISURE DI CENTRALITÀ

[DEGREE CENTRALITY] - Artisti più connessi:
  Guè: 0.5758
  Gemitaiz: 0.4949
  Night Skinny: 0.4444
  Emis Killa: 0.4444
  Fabri Fibra: 0.3939

[BETWEENNESS CENTRALITY] - Artisti bridge tra comunità:
  Guè: 0.0760
  Gemitaiz: 0.0527
  Clementino: 0.0431
  Night Skinny: 0.0422
  Elisa: 0.0413

[CLOSENESS CENTRALITY] - Artisti centrali nella rete:
  Guè: 0.6923
  Gemitaiz: 0.6429
  Night Skinny: 0.6266
  Emis Killa: 0.6226
  Fabri Fibra: 0.6149

[EIGENVECTOR CENTRALITY] - Artisti connessi ad altri importanti:
  Guè: 0.2500
  Gemitaiz: 0.2205
  Emis Killa: 0.2040
  Night Skinny: 0.2018
  Jake La Furia: 0.1842


In [33]:
print("\n" + "="*60)
print("4. RILEVAMENTO COMUNITÀ")
print("="*60)

# Algoritmo Louvain per community detection
partition = community_louvain.best_partition(G_main)

# Aggiungi community come attributo
nx.set_node_attributes(G_main, partition, 'community')

# Analisi comunità
num_communities = len(set(partition.values()))
print(f"\nNumero di comunità rilevate: {num_communities}")

# Dimensione comunità
community_sizes = Counter(partition.values())
print(f"\nDistribuzione dimensioni comunità:")
for comm_id, size in sorted(community_sizes.items(), key=lambda x: x[1], reverse=True)[:10]:
    print(f"  Comunità {comm_id}: {size} artisti")

# Modularity
modularity = community_louvain.modularity(partition, G_main)
print(f"\nModularità: {modularity:.4f}")
print(f"  → {'Forte' if modularity > 0.4 else 'Moderata' if modularity > 0.3 else 'Debole'} struttura a comunità")

# Artisti per comunità (top 3 comunità più grandi)
print(f"\nArtisti principali per comunità (top 3 comunità):")
for comm_id, size in sorted(community_sizes.items(), key=lambda x: x[1], reverse=True)[:3]:
    print(f"\n  Comunità {comm_id} ({size} artisti):")
    comm_nodes = [n for n, c in partition.items() if c == comm_id]
    comm_degrees = [(n, G_main.degree(n)) for n in comm_nodes]
    top_in_comm = sorted(comm_degrees, key=lambda x: x[1], reverse=True)[:5]
    for node, deg in top_in_comm:
        name = G_main.nodes[node].get('name', node)
        print(f"    - {name} ({deg} collab.)")



4. RILEVAMENTO COMUNITÀ

Numero di comunità rilevate: 6

Distribuzione dimensioni comunità:
  Comunità 5: 22 artisti
  Comunità 2: 18 artisti
  Comunità 1: 18 artisti
  Comunità 0: 16 artisti
  Comunità 4: 15 artisti
  Comunità 3: 11 artisti

Modularità: 0.2205
  → Debole struttura a comunità

Artisti principali per comunità (top 3 comunità):

  Comunità 5 (22 artisti):
    - Guè (57 collab.)
    - Izi (28 collab.)
    - Tedua (28 collab.)
    - Sick Luke (27 collab.)
    - Ernia (25 collab.)

  Comunità 2 (18 artisti):
    - Gemitaiz (49 collab.)
    - Fabri Fibra (39 collab.)
    - Jake La Furia (38 collab.)
    - MadMan (34 collab.)
    - Lazza (32 collab.)

  Comunità 1 (18 artisti):
    - Emis Killa (44 collab.)
    - Don Joe (35 collab.)
    - Shablo (23 collab.)
    - J-AX (23 collab.)
    - Fedez (20 collab.)


In [34]:
print("\n" + "="*60)
print("5. BRIDGE ANALYSIS - CONNETTORI TRA COMUNITÀ")
print("="*60)

# Identifica edge betweenness
edge_betweenness = nx.edge_betweenness_centrality(G_main)

# Top bridge edges
top_bridges = sorted(edge_betweenness.items(), key=lambda x: x[1], reverse=True)[:10]

print("\nTop 10 collaborazioni-bridge:")
for i, ((u, v), score) in enumerate(top_bridges, 1):
    name_u = G_main.nodes[u].get('name', u)
    name_v = G_main.nodes[v].get('name', v)
    comm_u = partition[u]
    comm_v = partition[v]
    is_bridge = "✓" if comm_u != comm_v else "✗"
    print(f"  {i}. {name_u} ↔ {name_v} (score: {score:.4f}) [Bridge: {is_bridge}]")

# Constraint (Burt's structural holes)
constraint = nx.constraint(G_main)
low_constraint = sorted(constraint.items(), key=lambda x: x[1])[:10]

print("\nArtisti con accesso a structural holes (basso constraint):")
for i, (node, const) in enumerate(low_constraint, 1):
    name = G_main.nodes[node].get('name', node)
    print(f"  {i}. {name} (constraint: {const:.4f})")



5. BRIDGE ANALYSIS - CONNETTORI TRA COMUNITÀ

Top 10 collaborazioni-bridge:
  1. Andrea Bocelli ↔ Elisa (score: 0.0268) [Bridge: ✗]
  2. Andrea Bocelli ↔ Ennio Morricone (score: 0.0200) [Bridge: ✗]
  3. Rocco Hunt ↔ Mario Biondi (score: 0.0178) [Bridge: ✓]
  4. Ornella Vanoni ↔ Jovanotti (score: 0.0135) [Bridge: ✗]
  5. Edoardo Bennato ↔ Clementino (score: 0.0134) [Bridge: ✓]
  6. Andrea Bocelli ↔ Tiziano Ferro (score: 0.0128) [Bridge: ✗]
  7. Fiorella Mannoia ↔ Achille Lauro (score: 0.0110) [Bridge: ✓]
  8. Ron ↔ Gigi D'Alessio (score: 0.0105) [Bridge: ✓]
  9. Clementino ↔ Jovanotti (score: 0.0097) [Bridge: ✓]
  10. M¥SS KETA ↔ Lo Stato Sociale (score: 0.0092) [Bridge: ✗]

Artisti con accesso a structural holes (basso constraint):
  1. Guè (constraint: 0.0523)
  2. Gemitaiz (constraint: 0.0558)
  3. Night Skinny (constraint: 0.0564)
  4. Emis Killa (constraint: 0.0582)
  5. Fabri Fibra (constraint: 0.0616)
  6. Jake La Furia (constraint: 0.0620)
  7. Don Joe (constraint: 0.0627)
  8.

In [35]:
print("\n" + "="*60)
print("6. CLIQUES E GRUPPI COESI")
print("="*60)

# Trova cliques massimali
cliques = list(nx.find_cliques(G_main))
clique_sizes = [len(c) for c in cliques]

print(f"\nNumero totale di cliques: {len(cliques)}")
print(f"Dimensione massima clique: {max(clique_sizes)}")
print(f"Dimensione media clique: {np.mean(clique_sizes):.2f}")

# Top 5 cliques più grandi
largest_cliques = sorted(cliques, key=len, reverse=True)[:5]
print(f"\nTop 5 cliques più grandi:")
for i, clique in enumerate(largest_cliques, 1):
    print(f"\n  Clique {i} ({len(clique)} artisti):")
    for node in clique[:10]:  # Mostra max 10
        name = G_main.nodes[node].get('name', node)
        print(f"    - {name}")

# K-core decomposition
k_cores = nx.core_number(G_main)
max_k = max(k_cores.values())

print(f"\nK-core massimo: {max_k}")
print(f"  → Esiste un nucleo di artisti con almeno {max_k} connessioni reciproche")

# Artisti nel k-core massimo
max_core_nodes = [n for n, k in k_cores.items() if k == max_k]
print(f"\nArtisti nel {max_k}-core ({len(max_core_nodes)} artisti):")
for node in sorted(max_core_nodes, key=lambda n: G_main.degree(n), reverse=True)[:10]:
    name = G_main.nodes[node].get('name', node)
    degree = G_main.degree(node)
    print(f"  - {name} ({degree} collab.)")



6. CLIQUES E GRUPPI COESI

Numero totale di cliques: 762
Dimensione massima clique: 9
Dimensione media clique: 5.15

Top 5 cliques più grandi:

  Clique 1 (9 artisti):
    - Guè
    - Gemitaiz
    - Jack The Smoker
    - Ensi
    - Night Skinny
    - Jake La Furia
    - Salmo
    - Noyz Narcos
    - Lazza

  Clique 2 (8 artisti):
    - Guè
    - Gemitaiz
    - Jack The Smoker
    - Ensi
    - Night Skinny
    - Jake La Furia
    - Salmo
    - MadMan

  Clique 3 (8 artisti):
    - Guè
    - Gemitaiz
    - Geolier
    - MadMan
    - Night Skinny
    - Marracash
    - Clementino
    - Coez

  Clique 4 (8 artisti):
    - Guè
    - Gemitaiz
    - Geolier
    - MadMan
    - Emis Killa
    - Marracash
    - Clementino
    - Coez

  Clique 5 (8 artisti):
    - Guè
    - Gemitaiz
    - Geolier
    - Noyz Narcos
    - Marracash
    - Night Skinny
    - Clementino
    - Coez

K-core massimo: 14
  → Esiste un nucleo di artisti con almeno 14 connessioni reciproche

Artisti nel 14-core (40 artisti)

In [36]:
print("\n" + "="*60)
print("7. PROPRIETÀ SMALL-WORLD")
print("="*60)

# Calcola metriche
C_real = nx.average_clustering(G_main)
if nx.is_connected(G_main):
    L_real = nx.average_shortest_path_length(G_main)
else:
    L_real = None

# Confronto con grafo random equivalente
n = G_main.number_of_nodes()
m = G_main.number_of_edges()
p = 2 * m / (n * (n - 1))

# Genera grafo random Erdős–Rényi
G_random = nx.erdos_renyi_graph(n, p, seed=42)
C_random = nx.average_clustering(G_random)

if nx.is_connected(G_random):
    L_random = nx.average_shortest_path_length(G_random)
else:
    # Se non connesso, usa componente gigante
    largest_cc = max(nx.connected_components(G_random), key=len)
    G_random_main = G_random.subgraph(largest_cc)
    L_random = nx.average_shortest_path_length(G_random_main)

print(f"\nRete musicale italiana:")
print(f"  Clustering: {C_real:.4f}")
if L_real:
    print(f"  Cammino medio: {L_real:.4f}")

print(f"\nGrafo random equivalente:")
print(f"  Clustering: {C_random:.4f}")
print(f"  Cammino medio: {L_random:.4f}")

# Test small-world: alto clustering + basso cammino medio
if L_real:
    sigma = (C_real / C_random) / (L_real / L_random)
    print(f"\nSmall-world coefficient (σ): {sigma:.4f}")
    if sigma > 1:
        print(f"  ✓ La rete ha proprietà SMALL-WORLD")
        print(f"    → Alto clustering locale + brevi distanze globali")
    else:
        print(f"  ✗ La rete non mostra forti proprietà small-world")



7. PROPRIETÀ SMALL-WORLD

Rete musicale italiana:
  Clustering: 0.3699
  Cammino medio: 2.0099

Grafo random equivalente:
  Clustering: 0.1830
  Cammino medio: 1.8440

Small-world coefficient (σ): 1.8546
  ✓ La rete ha proprietà SMALL-WORLD
    → Alto clustering locale + brevi distanze globali


In [46]:
print("\n" + "="*60)
print("8. ANALISI PER GENERE MUSICALE")
print("="*60)

# Verifica se esiste campo genere
if 'genres' in G_main.nodes[list(G_main.nodes())[0]]:
    
    # Estrai generi principali
    genre_dict = {}
    for node in G_main.nodes():
        genres = G_main.nodes[node].get('genres', [])
        if isinstance(genres, list) and len(genres) > 0:
            # Prendi primo genere
            genre_dict[node] = genres[0]
        else:
            genre_dict[node] = 'Unknown'
    
    # Distribuzione generi
    genre_count = Counter(genre_dict.values())
    print(f"\nDistribuzione generi (top 10):")
    for genre, count in genre_count.most_common(10):
        print(f"  {genre}: {count} artisti")
    
    # Assortatività per genere
    nx.set_node_attributes(G_main, genre_dict, 'genre')
    
    # Collaborazioni intra vs inter-genere
    intra_genre = 0
    inter_genre = 0
    for u, v in G_main.edges():
        if genre_dict[u] == genre_dict[v]:
            intra_genre += 1
        else:
            inter_genre += 1
    
    total_edges = intra_genre + inter_genre
    print(f"\nCollaborazioni intra-genere: {intra_genre} ({100*intra_genre/total_edges:.1f}%)")
    print(f"Collaborazioni inter-genere: {inter_genre} ({100*inter_genre/total_edges:.1f}%)")
    
    if intra_genre > inter_genre:
        print("  → Gli artisti tendono a collaborare dentro lo stesso genere")
    else:
        print("  → Forte cross-pollination tra generi diversi")
    
else:
    print("\nAttributo 'genres' non disponibile nel dataset")



8. ANALISI PER GENERE MUSICALE

Distribuzione generi (top 10):
  italian hip hop: 63 artisti
  italian adult pop: 13 artisti
  italian pop: 5 artisti
  classic italian pop: 4 artisti
  drill italiana: 2 artisti
  indie liguria: 2 artisti
  canzone d'autore: 1 artisti
  edm: 1 artisti
  trap italiana: 1 artisti
  italian indie pop: 1 artisti

Collaborazioni intra-genere: 578 (63.0%)
Collaborazioni inter-genere: 340 (37.0%)
  → Gli artisti tendono a collaborare dentro lo stesso genere


In [38]:
# Verifica attributi di un nodo casuale
sample_node = list(G_main.nodes())[0]
print(f"Attributi del nodo {sample_node}:")
print(G_main.nodes[sample_node])

# Verifica campo genres nel dataframe originale
print("\nGeneri nel dataframe nodes_ita:")
print(nodes_ita['genres'].head(10))
print(f"\nTipo del campo genres: {type(nodes_ita['genres'].iloc[0])}")


Attributi del nodo 4MR6tQyIrWK82b56cYPBDv:
{'name': 'Ornella Vanoni', 'followers': 152520.0, 'popularity': 52, 'genres': "['classic italian pop', 'italian adult pop']", 'chart_hits': "['it (1)']", 'community': 0, 'genre': 'Unknown'}

Generi nel dataframe nodes_ita:
152                 ['italian hip hop', 'trap italiana']
168    ["canzone d'autore", 'classic italian pop', 'i...
253    ['italian hip hop', 'italian indie pop', 'ital...
275            ['italian adult pop', 'italian pop rock']
301    ['italian hip hop', 'italian pop', 'pop virale...
314                                    ['trap italiana']
316    ['italian adult pop', 'italian hip hop', 'ital...
317                                      ['italian pop']
348                 ['italian hip hop', 'trap italiana']
543         ['classic italian pop', 'italian adult pop']
Name: genres, dtype: object

Tipo del campo genres: <class 'str'>
